In [1]:
import math
from nltk.corpus import wordnet
import nltk
from collections import Counter, defaultdict
import sys
import re
import numpy as np
from numpy.linalg import norm
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
from sklearn.metrics.scorer import make_scorer
from sklearn.neural_network import MLPRegressor

In [2]:
nltk.download('wordnet_ic')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /home/shivam/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shivam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/shivam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shivam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import gensim
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas
#% matplotlib inline

In [4]:
eng_vec_path = '../../../GoogleNews-vectors-negative300.bin'


In [5]:
word_vec = gensim.models.KeyedVectors.load_word2vec_format(eng_vec_path, binary=True, unicode_errors='ignore')

In [6]:

class Sim:
    def __init__(self, words, vectors):
        self.word_to_idx = {a: b for b, a in
                            enumerate(w.strip() for w in open(words))}
        self.mat = np.loadtxt(vectors)

    def bow_vec(self, b):
        vec = np.zeros(self.mat.shape[1])
        for k, v in b.items():
            idx = self.word_to_idx.get(k, -1)
            if idx >= 0:
                vec += self.mat[idx] / (norm(self.mat[idx]) + 1e-8) * v
        return vec

    def calc(self, b1, b2):
        v1 = self.bow_vec(b1)
        v2 = self.bow_vec(b2)
        return abs(v1.dot(v2) / (norm(v1) + 1e-8) / (norm(v2) + 1e-8))

stopwords = set([
"i", "a", "about", "an", "are", "as", "at", "be", "by", "for", "from",
"how", "in", "is", "it", "of", "on", "or", "that", "the", "this", "to",
"was", "what", "when", "where", "who", "will", "with", "the", "'s", "did",
"have", "has", "had", "were", "'ll"
])

nyt_sim = Sim('nyt_words.txt', 'nyt_word_vectors.txt')
wiki_sim = Sim('wikipedia_words.txt', 'wikipedia_word_vectors.txt')

def fix_compounds(a, b):
    sb = set(x.lower() for x in b)

    a_fix = []
    la = len(a)
    i = 0
    while i < la:
        if i + 1 < la:
            comb = a[i] + a[i + 1]
            if comb.lower() in sb:
                a_fix.append(a[i] + a[i + 1])
                i += 2
                continue
        a_fix.append(a[i])
        i += 1
    return a_fix

def load_data(path):
    sentences_pos = []
    r1 = re.compile(r'\<([^ ]+)\>')
    r2 = re.compile(r'\$US(\d)')
    for l in open(path):
        #l = l.decode('utf-8')
        l = l.replace(u'’', "'")
        l = l.replace(u'``', '"')
        l = l.replace(u"''", '"')
        l = l.replace(u"—", '--')
        l = l.replace(u"–", '--')
        l = l.replace(u"´", "'")
        l = l.replace(u"-", " ")
        l = l.replace(u"/", " ")
        l = r1.sub(r'\1', l)
        l = r2.sub(r'$\1', l)
        s = l.strip().split('\t')
        sa, sb = tuple(nltk.word_tokenize(s)
                          for s in l.strip().split('\t'))
        '''sa, sb = ([x.encode('utf-8') for x in sa],
                  [x.encode('utf-8') for x in sb])'''

        for s in (sa, sb):
            for i in range(len(s)):
                if s[i] == "n't":
                    s[i] = "not"
                elif s[i] == "'m":
                    s[i] = "am"
        sa, sb = fix_compounds(sa, sb), fix_compounds(sb, sa)
        sentences_pos.append((nltk.pos_tag(sa), nltk.pos_tag(sb)))
    return sentences_pos

def load_wweight_table(path):
    lines = open(path).readlines()
    wweight = defaultdict(float)
    if not len(lines):
        return (wweight, 0.)
    totfreq = int(lines[0])
    for l in lines[1:]:
        w, freq = l.split()
        freq = float(freq)
        if freq < 10:
            continue
        wweight[w] = math.log(totfreq / freq)

    return wweight

wweight = load_wweight_table('word-frequencies.txt')
minwweight = min(wweight.values())

def len_compress(l):
    return math.log(1. + l)

to_wordnet_tag = {
        'NN':wordnet.NOUN,
        'JJ':wordnet.ADJ,
        'VB':wordnet.VERB,
        'RB':wordnet.ADV
    }

word_matcher = re.compile('[^0-9,.(=)\[\]/_`]+$')
def is_word(w):
    return word_matcher.match(w) is not None

def get_locase_words(spos):
    return [x[0].lower() for x in spos
            if is_word(x[0])]

def make_ngrams(l, n):
    rez = [l[i:(-n + i + 1)] for i in range(n - 1)]
    rez.append(l[n - 1:])
    return list(zip(*rez))

def wvec_sim(la,lb):
    veca=np.zeros(300)
    vecb=np.zeros(300)
    m=0
    s=1
    for i in la:
        try:
            veca+= word_vec[i]/norm(word_vec[i]+ 1e-8)
        except KeyError:
            veca+= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)
    for i in lb:
        try:
            vecb+= word_vec[i]/norm(word_vec[i] + 1e-8)
        except:
            vecb+= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)
        
    return abs(veca.dot(vecb) / (norm(veca) + 1e-8 ) / (norm(vecb) + 1e-8))

def weighted_wvec_sim(lca, lcb):
    wa = Counter(lca)
    wb = Counter(lcb)
    wa = {x: wweight[x] * wa[x] for x in wa}
    wb = {x: wweight[x] * wb[x] for x in wb}
    
    veca=np.zeros(300)
    vecb=np.zeros(300)
    m=0
    s=1
    
    for k,v in wa.items():
        #print(k,v)
        try:
             veca+= word_vec[k]/norm(word_vec[k]+ 1e-8)*v
        except KeyError:
            print(k)
            veca+= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)*v

    for k,v in wb.items():
        #print(k,v)
        try:
             vecb+= word_vec[k]/norm(word_vec[k]+ 1e-8)*v
        except KeyError:
            print(k)
            vecb+= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)*v
        
    return  abs(veca.dot(vecb) / (norm(veca) + 1e-8 ) / (norm(vecb) + 1e-8))

def dist_sim(sim, la, lb):
    wa = Counter(la)
    wb = Counter(lb)
    d1 = {x:1 for x in wa}
    d2 = {x:1 for x in wb}
    return sim.calc(d1, d2)

def weighted_dist_sim(sim, lca, lcb):
    wa = Counter(lca)
    wb = Counter(lcb)
    wa = {x: wweight[x] * wa[x] for x in wa}
    wb = {x: wweight[x] * wb[x] for x in wb}
    return sim.calc(wa, wb)

def weighted_word_match(lca, lcb):
    wa = Counter(lca)
    wb = Counter(lcb)
    wsuma = sum(wweight[w] * wa[w] for w in wa)
    wsumb = sum(wweight[w] * wb[w] for w in wb)
    wsum = 0.

    for w in wa:
        wd = min(wa[w], wb[w])
        wsum += wweight[w] * wd
    p = 0.
    r = 0.
    if wsuma > 0 and wsum > 0:
        p = wsum / wsuma
    if wsumb > 0 and wsum > 0:
        r = wsum / wsumb
    f1 = 2 * p * r / (p + r) if p + r > 0 else 0.
    return f1

wpathsimcache = {}
def wpathsim(a, b):
    
    if a > b:
        b, a = a, b
    p = (a, b)
    if p in wpathsimcache:
        return wpathsimcache[p]
    if a == b:
        wpathsimcache[p] = 1.
        return 1.
    sa = wordnet.synsets(a)
    sb = wordnet.synsets(b)
    '''print(a)
    print(sa)
    print(b)
    print(sb)'''
    mx = max([wa.path_similarity(wb)
              for wa in sa
              for wb in sb if wa.path_similarity(wb) is not None ] + [0.]  )
    wpathsimcache[p] = mx
    return mx

wpathsimcache_jc = {}
def wpathsim_jc(a, b):
    if a > b:
        b, a = a, b
    p = (a, b)
    if p in wpathsimcache_jc:
        return wpathsimcache_jc[p]
    if a == b:
        wpathsimcache_jc[p] = 1.
        return 1.
    sa = wordnet.synsets(a)
    sb = wordnet.synsets(b)
    '''print(a)
    print(sa)
    print(b)
    print(sb)'''
    #print(a, b)
    mx_list=[]
    for wa in sa:
        for wb in sb:
            if (wa.pos()=='s' or wb.pos()=='s'):
                continue
            if (wa.pos()=='a' or wb.pos()=='a'):
                continue
            
            if (wa.pos()=='r' or wb.pos()=='r'):
                continue
            
            if (wa.pos() == wb.pos()):
                t=wa.jcn_similarity(wb,semcor_ic)
                if(t<0.001):
                    t1=0.001
                    mx_list.append(t1)
                elif (t>1):
                    t1=1
                    mx_list.append(t1)
                else:
                    mx_list.append(wa.jcn_similarity(wb,semcor_ic))
    mx_list=mx_list +[0.0]
    mx= max(mx_list)
    wpathsimcache_jc[p] = mx
    return mx

wpathsimcache_lin = {}
def wpathsim_lin(a, b):
    if a > b:
        b, a = a, b
    p = (a, b)
    if p in wpathsimcache_lin:
        return wpathsimcache_lin[p]
    if a == b:
        wpathsimcache_lin[p] = 1.
        return 1.
    sa = wordnet.synsets(a)
    sb = wordnet.synsets(b)
    '''print(a)
    print(sa)
    print(b)
    print(sb)'''
    #print(a, b)
    mx_list=[]
    for wa in sa:
        for wb in sb:
            if (wa.pos()=='s' or wb.pos()=='s'):
                continue
            if (wa.pos()=='a' or wb.pos()=='a'):
                continue
            
            if (wa.pos()=='r' or wb.pos()=='r'):
                continue
            
            if (wa.pos() == wb.pos()):
                t=wa.jcn_similarity(wb,semcor_ic)
                if(t<0.001):
                    t1=0.001
                    mx_list.append(t1)
                elif (t>1):
                    t1=1
                    mx_list.append(t1)
                else:
                    mx_list.append(wa.lin_similarity(wb,semcor_ic))
    mx_list=mx_list +[0.0]
    mx= max(mx_list)
    wpathsimcache_lin[p] = mx
    return mx

def calc_wn_prec(lema, lemb,val):
    rez = 0.
    for a in lema:
        ms = 0.
        for b in lemb:
            if(val==0):
                ms = max(ms, wpathsim(a, b))
            elif(val==1):
                ms = max(ms, wpathsim_jc(a, b))
            else:
                ms = max(ms, wpathsim_lin(a, b))
                
        rez += ms
    return rez / len(lema)

def wn_sim_match(lema, lemb,val):
    f1 = 1.
    p = 0.
    r = 0.
    if len(lema) > 0 and len(lemb) > 0:
        p = calc_wn_prec(lema, lemb,val)
        r = calc_wn_prec(lemb, lema,val)
        f1 = 2. * p * r / (p + r) if p + r > 0 else 0.
    return f1

def ngram_match(sa, sb, n):
    nga = make_ngrams(sa, n)
    ngb = make_ngrams(sb, n)
    matches = 0
    c1 = Counter(nga)
    for ng in ngb:
        if c1[ng] > 0:
            c1[ng] -= 1
            matches += 1
    p = 0.
    r = 0.
    f1 = 1.
    if len(nga) > 0 and len(ngb) > 0:
        p = matches / float(len(nga))
        r = matches / float(len(ngb))
        f1 = 2 * p * r / (p + r) if p + r > 0 else 0.
    return f1

def get_lemmatized_words(sa):
    rez = []
    for w, wpos in sa:
        w = w.lower()
        if w in stopwords or not is_word(w):
            continue
        wtag = to_wordnet_tag.get(wpos[:2])
        if wtag is None:
            wlem = w
        else:
            wlem = wordnet.morphy(w, wtag) or w
        rez.append(wlem)
    return rez

def is_stock_tick(w):
    return w[0] == '.' and len(w) > 1 and w[1:].isupper()

def stocks_matches(sa, sb):
    ca = set(x[0] for x in sa if is_stock_tick(x[0]))
    cb = set(x[0] for x in sb if is_stock_tick(x[0]))
    isect = len(ca.intersection(cb))
    la = len(ca)
    lb = len(cb)

    f = 1.
    if la > 0 and lb > 0:
        if isect > 0:
            p = float(isect) / la
            r = float(isect) / lb
            f = 2 * p * r / (p + r)
        else:
            f = 0.
    return (len_compress(la + lb), f)

def case_matches(sa, sb):
    ca = set(x[0] for x in sa[1:] if x[0][0].isupper()
            and x[0][-1] != '.')
    cb = set(x[0] for x in sb[1:] if x[0][0].isupper()
            and x[0][-1] != '.')
    la = len(ca)
    lb = len(cb)
    isect = len(ca.intersection(cb))

    f = 1.
    if la > 0 and lb > 0:
        if isect > 0:
            p = float(isect) / la
            r = float(isect) / lb
            f = 2 * p * r / (p + r)
        else:
            f = 0.
    return (len_compress(la + lb), f)

risnum = re.compile(r'^[0-9,./-]+$')
rhasdigit = re.compile(r'[0-9]')

def match_number(xa, xb):
    if xa == xb:
        return True
    xa = xa.replace(',', '')
    xb = xb.replace(',', '')

    try:
        va = int(float(xa))
        vb = int(float(xb))
        if (va == 0 or vb == 0) and va != vb:
            return False
        fxa = float(xa)
        fxb = float(xb)
        if abs(fxa - fxb) > 1:
            return False
        diga = xa.find('.')
        digb = xb.find('.')
        diga = 0 if diga == -1 else len(xa) - diga - 1
        digb = 0 if digb == -1 else len(xb) - digb - 1
        if diga > 0 and digb > 0 and va != vb:
            return False
        dmin = min(diga, digb)
        if dmin == 0:
            if abs(round(fxa, 0) - round(fxb, 0)) < 1e-5:
                return True
            return va == vb
        return abs(round(fxa, dmin) - round(fxb, dmin)) < 1e-5
    except:
        pass

    return False

def number_features(sa, sb):
    numa = set(x[0] for x in sa if risnum.match(x[0]) and
            rhasdigit.match(x[0]))
    numb = set(x[0] for x in sb if risnum.match(x[0]) and
            rhasdigit.match(x[0]))
    isect = 0
    for na in numa:
        if na in numb:
            isect += 1
            continue
        for nb in numb:
            if match_number(na, nb):
                isect += 1
                break

    la, lb = len(numa), len(numb)

    f = 1.
    subset = 0.
    if la + lb > 0:
        if isect == la or isect == lb:
            subset = 1.
        if isect > 0:
            p = float(isect) / la
            r = float(isect) / lb
            f = 2. * p * r / (p + r)
        else:
            f = 0.
    return (len_compress(la + lb), f, subset)

def relative_len_difference(lca, lcb):
    la, lb = len(lca), len(lcb)
    return abs(la - lb) / float(max(la, lb) + 1e-5)

def relative_ic_difference(lca, lcb):
    #wa = sum(wweight[x] for x in lca)
    #wb = sum(wweight[x] for x in lcb)
    wa = sum(max(0., wweight[x] - minwweight) for x in lca)
    wb = sum(max(0., wweight[x] - minwweight) for x in lcb)
    return abs(wa - wb) / (max(wa, wb) + 1e-5)

def my_custom_function(y_true, y_predict):
    corr,_ = pearsonr(y_true, y_predict)
    return corr

    
 #   gs = GridSearchCV(clf, param_grid={'kernel': [ 'rbf'],'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='roc_auc')
    
    
    


In [7]:
def post_process(path):
    import sys


    orig = []
    for l in open('../../test-gold/STS.input.surprise.SMTnews.txt'):
        orig.append([x.strip() for x in l.lower().split("\t")])

    scores = list(map(float, open(path).readlines()))

    if len(orig) != len(scores):
        print (sys.stderr, "Error: inputs should have the same number of lines")
        exit(1)
    processed=[]
    f = open('processed_'+path, 'w')
    for i, s in enumerate(scores):
        if orig[0] == orig[1]:
            s = 5.
        if s > 5:
            s = 5.
        if s < 0:
            s = 0.
        processed.append(s)
        #print (s)
        f.write("%s\n" % s)
    f.close()
    return processed

## takelab_score

In [10]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,0),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            #wvec_sim(lema,lemb),
            #weighted_wvec_sim(lema,lemb)
            dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            weighted_dist_sim(nyt_sim, lema, lemb),
            weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f



if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    

/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [11]:
data.shape

(2234, 21)

In [12]:
with open('takelab_output.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()
model_pred=np.array(post_process('takelab_output.txt'))


corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.47930463231775344


### Pearson: 0.88224

## Using JC Similarity

In [13]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,1),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            #wvec_sim(lema,lemb),
            #weighted_wvec_sim(lema,lemb)
            dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            weighted_dist_sim(nyt_sim, lema, lemb),
            weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f

if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    
    

/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [14]:
with open('takelab_jc.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()

model_pred=np.array(post_process('takelab_jc.txt'))
corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.42373502041138283


RESULT: (Pearson: 0.87477)

## Using Lin Similarity

In [15]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,2),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            #wvec_sim(lema,lemb),
            #weighted_wvec_sim(lema,lemb)
            dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            weighted_dist_sim(nyt_sim, lema, lemb),
            weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f

if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    
    

/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
with open('takelab_lin.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()
model_pred=np.array(post_process('takelab_lin.txt'))
corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.43199889357383714


RESULT:(Pearson: 0.87057)


## word2vec with weighted_LSA_vec 


In [17]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,0),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            wvec_sim(lema,lemb),
            #weighted_wvec_sim(lema,lemb)
            #dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            weighted_dist_sim(nyt_sim, lema, lemb),
            weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f

if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    
    

/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [18]:
with open('wv_wlsa.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()
model_pred=np.array(post_process('wv_wlsa.txt'))
corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.504267809977364


RESULT: Pearson Correlation: ( 0.8808)

## JC AND LIN Together

In [8]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,1),
            wn_sim_match(lema, lemb,2),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            #wvec_sim(lema,lemb),
            #weighted_wvec_sim(lema,lemb),
            dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            #weighted_dist_sim(nyt_sim, lema, lemb),
            #weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f

if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    

/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [9]:
with open('jc_lin.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()
model_pred=np.array(post_process('jc_lin.txt'))
corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.40683524838870855


RESULT:(Pearson: 0.87136)

## word2vec with weighted_word2vec

In [10]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,0),
           # wn_sim_match(lema, lemb,1),
            #wn_sim_match(lema, lemb,2),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            wvec_sim(lema,lemb),
            weighted_wvec_sim(lema,lemb),
            #dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            #weighted_dist_sim(nyt_sim, lema, lemb),
            #weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f

if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    

and
and
and
and
and
and
and
and
axe
axe
and
and
and
and
and
and
and
and
and
and
and
and
swinge
swinge
and
and
and
and
and
and
and
and
and
and
and
and
swinge
and
and
and
oots
and
and
and
and
and
and
and
and
seadoo
and
and
and
and
descale
swinge
and
and
and
axe
and
and
and
and
and
and
and
and
and
and
and
and
and
and
and
swinge
and
and
and
episcopalian
episcopalian
amgen
allergan
and
and
''
''
webvpn
smarnet
and
and
dallager
dallager
''
karanja
karanja
and
and
swartz
swartz
and
and
españa
and
espaa
and
plofsky
plofsky
frankel
and
hilsenrath
and
klarman
klarman
and
and
''
and
episcopalian
and
episcopalian
squyres
athena
squyres
'
and
and
and
''
and
bucklew
bucklew
shevaun
pennington
studabaker
shevaun
pennington
studabaker
and
and
waksal
and
waksal
and
and
zambrano
and
''
''
''
and
'
vba
studdard
aiken
studdard
aiken
and
''
''
and
imclone
erbitux
and
imclone
sensenbrenner
sensenbrenner
and
and
and
heatley
and
heatley
and
strayhorn
strayhorn
sahel
ntsb
nhtsa
and
jazeera
jazeera
franklyn
ber

lurd
ja'neh
and
kedo
:
''
tiburtina
and
tiburtina
and
worldcom
and
worldcom
and
and
and
and
bloomfield
taubman
cgt
and
cgt
and
and
and
benning
benning
gambian
gambian
roush
and
diarrhoea
and
kiernan
seifert
seifert
mccloskey
mccloskey
and
ijaw
and
ijaw
and
and
and
and
and
and
and
centre
and
and
centre
and
'
marín
galicia
and
marín
galicia
and
and
tunisian
tunisian
and
programme
programme
and
'
and
organise
and
and
meps
and
and
honourable
and
programme
programme
and
and
and
neighbourly
and
and
and
?
and
?
and
and
lalumière
'
and
lalumière
and
and
and
and
and
and
and
centre
and
and
centre
and
d'impact
and
and
and
;
and
and
and
d'impact
and
and
and
and
and
and
and
and
and
'
and
and
and
and
and
and
programme
programme
and
gayssot
gayssot
and
and
and
and
labour
igc
favourable
igc
ijaw
and
ijaw
and
and
and
and
and
and
and
'
and
organise
and
kostunica
and
and
and
and
and
and
and
?
?
plough
kostunica
and
koitunica
and
labelling
and
labelling
and
and
favour
favour
himara
democratisation
and
alb

and
and
and
and
and
and
:
kostunica
democratisation
and
:
kostunica
;
democratisation
and
and
lalumière
'
and
lalumière
and
and
and
and
and
and
and
and
himara
democratisation
and
albania
himara
democratisation
and
albania
labour
igc
labour
favourable
igc
garcía
margallo
marfil
favour
garcía
margallo
marfil
and
and
and
?
‘
'model
and
?
and
rã©crã©ationnelles
garcía
margallo
marfil
favour
garcía
margallo
marfil
and
defence
and
programme
programme
;
unipersonal
'
and
and
and
and
and
and
labelling
and
labelling
and
tunisian
tunisian
:
:
favour
and
favour
karamanou
'
and
'
and
karamanou
and
?
?
and
and
and
and
and
:
and
:
tunisian
tunisian
and
and
:
kostunica
democratisation
and
:
koitunica
;
democratisation
and
and
and


/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


'
and
ahmadinejad
;
and
ahmadinejad
destabilise
and
''
and
''
''
muybarak
''
muybarak
'
and
paneuropéenne
?
?
?
?
''
'only
and
and
and
indonésiens
and
and
and
and
and
and
and
and
and
ahmadinejad
;
and
ahmadinejad
and
''
“
”
and
and
tocqueville
tocqueville
alstom
and
alstom
and
and
?
?
and
alstom
and
alstom
and
''
muybarak
''
gauchet
gauchet
and
and
''
and
?
?
and
and
institutionalisation
ahmadinejad
;
ahmadinejad
and
and
institutionalisation
and
and
islamiste
and
and
gauchet
gauchet
?
?
and
and
and
and
and
and
and
and
?
donnerait
?
and
and
and
ahmadinejad
;
and
ahmadinejad
''
‘
'
and
and
khamenei
and
particuli�rement
supr�me
khamenei
and
and
''
tocqueville
tocqueville
tocqueville
tocqueville
and
and
politicisation
hongkong
and
hongkong
and
''
muybarak
“
”
and
and
and
and
and
and
and
?
?
?
?
and
and
khamenei
and
particuli
khamenei
and
and
and
and
khamenei
and
particuli
rement
supr
khamenei
tocqueville
tocqueville
and
ahmadinejad
;
and
ahmadinejad
and
''
and
‘
'
nombrilistes
and
and
and


In [11]:
with open('wv_wwv.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()
model_pred=np.array(post_process('wv_wwv.txt'))
corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.5014453466636745


Result: (Pearson: 0.88384)

## word2vec_wwv_jc_lin

In [12]:
def calc_features(sa, sb):
    olca = get_locase_words(sa)
    olcb = get_locase_words(sb)
    lca = [w for w in olca if w not in stopwords]
    lcb = [w for w in olcb if w not in stopwords]
    lema = get_lemmatized_words(sa)
    lemb = get_lemmatized_words(sb)
   
    f = []
    f += number_features(sa, sb)
    f += case_matches(sa, sb)
    f += stocks_matches(sa, sb)
   
    f += [
            ngram_match(lca, lcb, 1),
            ngram_match(lca, lcb, 2),
            ngram_match(lca, lcb, 3),
            ngram_match(lema, lemb, 1),
            ngram_match(lema, lemb, 2),
            ngram_match(lema, lemb, 3),
            wn_sim_match(lema, lemb,1),
            wn_sim_match(lema, lemb,2),
            weighted_word_match(olca, olcb),
            weighted_word_match(lema, lemb),
            wvec_sim(lema,lemb),
            weighted_wvec_sim(lema,lemb),
            #dist_sim(nyt_sim, lema, lemb),
            #dist_sim(wiki_sim, lema, lemb),
            #weighted_dist_sim(nyt_sim, lema, lemb),
            #weighted_dist_sim(wiki_sim, lema, lemb),
            relative_len_difference(lca, lcb),
            relative_ic_difference(olca, olcb)
        ]

    return f

if __name__ == "__main__":
    
 
    data=[]
    c=0
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../train/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]
        
    '''for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRvid_clean.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.MSRpar.txt')):
        data+=[(calc_features(*sp))]
        
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.SMTeuroparl.txt')):
        data+=[(calc_features(*sp))]'''
        
    data=np.array(data)   
    
    f= open('../../train/STS.gs.MSRvid.txt')
    l1= f.readlines()
    l1=[float(w.strip()) for w in l1]
    
    f= open('../../train/STS.gs.MSRpar.txt')
    l2= f.readlines()
    l2=[float(w.strip()) for w in l2]
    
    f= open('../../train/STS.gs.SMTeuroparl.txt')
    l3= f.readlines()
    l3=[float(w.strip()) for w in l3]
    
    '''f= open('../../test-gold/STS.gs.MSRvid.txt')
    l4= f.readlines()
    l4=[float(w.strip()) for w in l4]
    
    f= open('../../test-gold/STS.gs.MSRpar.txt')
    l5= f.readlines()
    l5=[float(w.strip()) for w in l5]
    
    f= open('../../test-gold/STS.gs.SMTeuroparl.txt')
    l6= f.readlines()
    l6=[float(w.strip()) for w in l6]'''
    
    label= l1 +l2 + l3
    
    label=np.array(label).reshape(len(label),)
    
    my_scorer = make_scorer(my_custom_function, greater_is_better=True)
    
    '''gs = GridSearchCV(MLPRegressor(), param_grid={'solver': ['lbfgs','sgd','adam'],'alpha': 10.0 ** -np.arange(1, 7),'learning_rate':['constant'], 'learning_rate_init': [0.001],'hidden_layer_sizes': [25,50,100],'activation': ['identity','relu','logistic', 'tanh']},scoring=my_scorer)'''
    
    gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring=my_scorer,cv=10)
    '''gs = GridSearchCV(SVR(kernel='rbf'), param_grid={'C': [1,2,5,10,20, 50,100,200,500,1000],'gamma':[2,1,.5,.2,.1,.05,.02,.01,.005,.002]},scoring='r2',cv=10)'''
    gs.fit(data,label)
    m=gs.best_estimator_
    model=m.fit(data,label)
    
    
    test_data=[]
    for idx, sp in enumerate(load_data('../../test-gold/STS.input.surprise.SMTnews.txt')):
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('../../test-gold/STS.gs.surprise.SMTnews.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)
    
    '''test_data=[]
    for idx, sp in enumerate(load_data('trial1/STS.input.txt')):
        y = 0. if scores is None else scores[idx]
        test_data+=[(calc_features(*sp))]
        
    test_data=np.array(test_data)
    
    f= open('trial1/STS.gs.txt')
    test_label= f.readlines()
    test_label=[float(w.strip()) for w in test_label]
    test_label=np.array(test_label).reshape(len(test_label),)'''
    
    predictions= model.predict(test_data)
    
    

and
and
and
and
and
and
and
and
axe
axe
and
and
and
and
and
and
and
and
and
and
and
and
swinge
swinge
and
and
and
and
and
and
and
and
and
and
and
and
swinge
and
and
and
oots
and
and
and
and
and
and
and
and
seadoo
and
and
and
and
descale
swinge
and
and
and
axe
and
and
and
and
and
and
and
and
and
and
and
and
and
and
and
swinge
and
and
and
episcopalian
episcopalian
amgen
allergan
and
and
''
''
webvpn
smarnet
and
and
dallager
dallager
''
karanja
karanja
and
and
swartz
swartz
and
and
españa
and
espaa
and
plofsky
plofsky
frankel
and
hilsenrath
and
klarman
klarman
and
and
''
and
episcopalian
and
episcopalian
squyres
athena
squyres
'
and
and
and
''
and
bucklew
bucklew
shevaun
pennington
studabaker
shevaun
pennington
studabaker
and
and
waksal
and
waksal
and
and
zambrano
and
''
''
''
and
'
vba
studdard
aiken
studdard
aiken
and
''
''
and
imclone
erbitux
and
imclone
sensenbrenner
sensenbrenner
and
and
and
heatley
and
heatley
and
strayhorn
strayhorn
sahel
ntsb
nhtsa
and
jazeera
jazeera
franklyn
ber

ijaw
and
ijaw
and
and
and
and
and
and
and
centre
and
and
centre
and
'
marín
galicia
and
marín
galicia
and
and
tunisian
tunisian
and
programme
programme
and
'
and
organise
and
and
meps
and
and
honourable
and
programme
programme
and
and
and
neighbourly
and
and
and
?
and
?
and
and
lalumière
'
and
lalumière
and
and
and
and
and
and
and
centre
and
and
centre
and
d'impact
and
and
and
;
and
and
and
d'impact
and
and
and
and
and
and
and
and
and
'
and
and
and
and
and
and
programme
programme
and
gayssot
gayssot
and
and
and
and
labour
igc
favourable
igc
ijaw
and
ijaw
and
and
and
and
and
and
and
'
and
organise
and
kostunica
and
and
and
and
and
and
and
?
?
plough
kostunica
and
koitunica
and
labelling
and
labelling
and
and
favour
favour
himara
democratisation
and
albania
himara
democratisation
and
albania
and
and
labelling
and
labelling
and
and
:
kostunica
democratisation
and
:
kostunica
;
democratisation
and
and
and
and
gayssot
gayssot
recognise
centre
and
and
centre
and
and
and
and
and
'
and
plough


and
and
and
and
and
labelling
and
labelling
and
tunisian
tunisian
:
:
favour
and
favour
karamanou
'
and
'
and
karamanou
and
?
?
and
and
and
and
and
:
and
:
tunisian
tunisian
and
and
:
kostunica
democratisation
and
:
koitunica
;
democratisation
and
and
and


/apps/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


'
and
ahmadinejad
;
and
ahmadinejad
destabilise
and
''
and
''
''
muybarak
''
muybarak
'
and
paneuropéenne
?
?
?
?
''
'only
and
and
and
indonésiens
and
and
and
and
and
and
and
and
and
ahmadinejad
;
and
ahmadinejad
and
''
“
”
and
and
tocqueville
tocqueville
alstom
and
alstom
and
and
?
?
and
alstom
and
alstom
and
''
muybarak
''
gauchet
gauchet
and
and
''
and
?
?
and
and
institutionalisation
ahmadinejad
;
ahmadinejad
and
and
institutionalisation
and
and
islamiste
and
and
gauchet
gauchet
?
?
and
and
and
and
and
and
and
and
?
donnerait
?
and
and
and
ahmadinejad
;
and
ahmadinejad
''
‘
'
and
and
khamenei
and
particuli�rement
supr�me
khamenei
and
and
''
tocqueville
tocqueville
tocqueville
tocqueville
and
and
politicisation
hongkong
and
hongkong
and
''
muybarak
“
”
and
and
and
and
and
and
and
?
?
?
?
and
and
khamenei
and
particuli
khamenei
and
and
and
and
khamenei
and
particuli
rement
supr
khamenei
tocqueville
tocqueville
and
ahmadinejad
;
and
ahmadinejad
and
''
and
‘
'
nombrilistes
and
and
and


In [13]:
with open('full.txt', 'w') as f:
        for item in predictions:
            f.write("%s\n" % item)
f.close()
model_pred=np.array(post_process('full.txt',))
corr,_ = pearsonr(test_label, model_pred)
print (corr)

0.47202732583536267


RESULT: (Pearson: 0.87911)

In [14]:
l='The bird is bathing in the sink.\tBirdie is washing itself in the water basin.'

In [15]:
sa, sb = tuple(nltk.word_tokenize(s)
                          for s in l.strip().split('\t'))

In [16]:
se=[]
se.append((nltk.pos_tag(sa), nltk.pos_tag(sb)))

In [17]:
se


[([('The', 'DT'),
   ('bird', 'NN'),
   ('is', 'VBZ'),
   ('bathing', 'VBG'),
   ('in', 'IN'),
   ('the', 'DT'),
   ('sink', 'NN'),
   ('.', '.')],
  [('Birdie', 'NNP'),
   ('is', 'VBZ'),
   ('washing', 'VBG'),
   ('itself', 'PRP'),
   ('in', 'IN'),
   ('the', 'DT'),
   ('water', 'NN'),
   ('basin', 'NN'),
   ('.', '.')])]

In [18]:
sa,sb=se[0]

In [19]:
lema = get_lemmatized_words(sa)
lemb = get_lemmatized_words(sb)

In [20]:
lema

['bird', 'bathe', 'sink']

In [21]:
lemb

['birdie', 'wash', 'itself', 'water', 'basin']

In [22]:
s=weighted_wvec_sim(lema,lemb)
s

0.5526149829037504

In [23]:
def wvec_sim(la,lb):
    veca=np.zeros(300)
    vecb=np.zeros(300)
    m=0
    s=1
    for i in la:
        try:
            veca+= word_vec[i]/norm(word_vec[i]+ 1e-8)
        except KeyError:
            print(i)
            veca+= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)
    for i in lb:
        try:
            vecb+= word_vec[i]/norm(word_vec[i] + 1e-8)
        except KeyError:
            print(i)
            vecb+= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)
        
    return abs(veca.dot(vecb) / (norm(veca) + 1e-8 ) / (norm(vecb) + 1e-8))

In [24]:
l=9


In [25]:
lca=lema
lcb=lemb

In [26]:
wordnet.morp

AttributeError: 'WordNetCorpusReader' object has no attribute 'morp'

In [27]:
h=7

In [ ]:
k={1:'d',2:'c',3:'g',4:'r',5:'t'}

In [ ]:
list(k)[:2]

In [ ]:
list(brown_ic['n'].items())[:5]

In [ ]:
k.value


In [ ]:
k.items()


In [ ]:
d=[]

In [ ]:
for i in range(0,9):
    d+=[[i]]

In [ ]:
d


In [ ]:
r=[54,5454,43,23]

In [ ]:
d=d+r

In [ ]:
d


In [28]:
word_vec['institutionalization']

array([-0.30078125,  0.12695312, -0.16601562,  0.27539062,  0.02270508,
       -0.00939941,  0.296875  , -0.22558594,  0.09326172,  0.10742188,
        0.01831055, -0.12988281, -0.03466797,  0.41015625,  0.140625  ,
        0.21679688,  0.37695312, -0.24511719, -0.26757812, -0.02758789,
        0.38867188, -0.08935547,  0.20214844,  0.0177002 , -0.00291443,
       -0.08056641, -0.12597656, -0.16894531, -0.04370117, -0.01544189,
       -0.16699219, -0.10449219,  0.02001953,  0.09082031, -0.01660156,
       -0.09960938,  0.30859375,  0.03637695, -0.16796875, -0.28515625,
        0.11572266,  0.2734375 ,  0.10205078, -0.19140625, -0.04174805,
       -0.37109375, -0.1640625 ,  0.10302734, -0.09570312,  0.0112915 ,
        0.11572266,  0.02966309, -0.578125  ,  0.11767578, -0.14160156,
        0.25195312, -0.23925781, -0.09179688, -0.21875   , -0.33789062,
       -0.359375  ,  0.04345703,  0.17382812, -0.19921875,  0.00631714,
        0.04907227, -0.01660156, -0.0088501 , -0.11572266, -0.08